**pythonに不慣れた人にもわかりやすいようにメモを追加しています。**<br>
<br>
ここでは、この後のtraining等で使いたいtableをつくることだけしています。



In [ ]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [ ]:
## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [ ]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
## Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

【メモ】<br>
evaluationが出てきたので、END_TRAINの数字が大きくなっている。
1913日分から１９４１日分

In [ ]:
########################### Vars
#################################################################################
TARGET = 'sales'         # Our main target
#END_TRAIN = 1913         # Last day in train set
END_TRAIN = 1941         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [ ]:
########################### Load Data
#################################################################################
print('Load Main Data')

# Here are reafing all our data 
# without any limitations and dtype modification
#train_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
train_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')
prices_df = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
calendar_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')

In [ ]:
train_df.head()

【メモ】<br>
メモリが足りなくなるので、古い３００日分のデータを落としている。<br>
上のheadを見るとわかるが、左から右へ６個分は必要なものなので、それをSkipして、３００日分データ削除。<br>
<br>
実務のデータ処理でも使いそうな技術です****<br>

In [ ]:
# 'sales_train_evaluation.csv' has extra 28 days, for memory exhaust, drop 1st XXXdays (dropping 28days was not enouth)
train_df.drop(train_df.columns[6:6+300], axis=1, inplace=True)
train_df

【メモ】<br>
上で、id,item_id,dept_id,cat_id,store_id,state_idは維持したまま、d_301からスタートしていることがわかります。

【メモ】<br>
下のmeltはこちらがわかりやすいです。<br>
https://qiita.com/ishida330/items/922caa7acb73c1540e28


In [ ]:
########################### Make Grid
#################################################################################
print('Create Grid')

# We can tranform horizontal representation 
# to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
# and labels are 'd_' coulmns

index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)



In [ ]:
grid_df.head()

【メモ】<br>
間にgrid_df.head()を入れるため、もとは一つだったCellを３つにぶった切りました。<br>
**維持したいIndexをそのままに、rowを増やす技術は実務でも有効と思います。**

In [ ]:
# If we look on train_df we se that 
# we don't have a lot of traning rows
# but each day can provide more train data
print('Train rows:', len(train_df), len(grid_df))

# To be able to make predictions
# we need to add "test set" to our grid
add_grid = pd.DataFrame()
for i in range(1,29):
    ## 上から参照用にコピー
    ## index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
    ## inde_columnsだけでdfを作る。
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    ## 上から参照用にコピー
    ## END_TRAIN = 1941   
    ## "d"columnに、d_1942,d_1943を追加
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    ## TARGET = 'sales' salesのからを準備 
    temp_df[TARGET] = np.nan
    ## つまり、d_1942,d_1943、、、で、salesが空のものを作っている。
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)



In [ ]:
grid_df.tail()

【メモ】<br>
上で、grid_dfの下側に、d_1942以降のものが、salesがNaNで****追加されていることがわかります。

In [ ]:
# Remove some temoprary DFs
del temp_df, add_grid

# We will not need original train_df
# anymore and can remove it
del train_df

# You don't have to use df = df construction
# you can use inplace=True instead.
# like this
# grid_df.reset_index(drop=True, inplace=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# We can free some memory 
# by converting "strings" to categorical
# it will not affect merging and 
# we will not lose any valuable data
for col in index_columns:
    grid_df[col] = grid_df[col].astype('category')

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

【メモ】<br>
prices_dfを扱うので、まずはじめに形を確認。

In [ ]:
prices_df.head(10)

【メモ】<br>
下のgroupbyは、まとまりごとになにか処理（この場合は、agg(["min"])）を行うときに使う。groupbyと一緒によく使うのがmean()<br>
groupbyはここがわかりやすい。<br>
https://deepage.net/features/pandas-groupby.html <br>
.aggについては、こちら。関数を実行しつつ、columnを付け加えるような動き？ <br>
http://sinhrks.hatenablog.com/entry/2014/10/13/005327

In [ ]:
########################### Product Release date
#################################################################################
print('Release week')

# It seems that leadings zero values
# in each train_df item row
# are not real 0 sales but mean
# absence for the item in the store
# we can safe some memory by removing
# such zeros

# Prices are set by week
# so it we will have not very accurate release week 
release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()


In [ ]:
release_df.head()

In [ ]:
release_df.columns = ['store_id','item_id','release']



In [ ]:
release_df.head()

【メモ】<br>
下のmerge_by_concatは、grid_dfとrelease_dfで["store_id","item_id"]が同じ行を探し、release_dfの行を、grid_dfにマージしている。

In [ ]:
# Now we can merge release_df
grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
del release_df


In [ ]:
grid_df.head()

In [ ]:
# We want to remove some "zeros" rows
# from grid_df 
# to do it we need wm_yr_wk column
# let's merge partly calendar_df to have it
grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk','d']], ['d'])
                      


In [ ]:
grid_df.head()

【メモ】<br>
wm_yr_wkとreleaseを比較し、wm_yr_wkのほうが大きい行だけ抜き出して、新しいgrid_dfを作っている。

In [ ]:
# Now we can cutoff some rows 
# and safe memory 
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']]
grid_df = grid_df.reset_index(drop=True)



In [ ]:
grid_df.head()

In [ ]:
# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# Should we keep release week 
# as one of the features?
# Only good CV can give the answer.
# Let's minify the release values.
# Min transformation will not help here 
# as int16 -> Integer (-32768 to 32767)
# and our grid_df['release'].max() serves for int16
# but we have have an idea how to transform 
# other columns in case we will need it
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

【メモ】<br>
to_pickleは、軽量DB化して保存している。複数形のpicklesはピクルスのこと。<br>よく酢漬けとか言われる。

In [ ]:
########################### Save part 1
#################################################################################
print('Save Part 1')

# We have our BASE grid ready
# and can save it as pickle file
# for future use (model training)
grid_df.to_pickle('grid_part_1.pkl')

print('Size:', grid_df.shape)

【メモ】<br>
prices_df['price_max']などは、新しいcolumn(特徴量 feature)を追加している。
groupbyを使っているので、"store_id"と"item_id"が同じもののmax/min/std/meanを、新しいcolumnに追加している。

In [ ]:
########################### Prices
#################################################################################
print('Prices')

# We can do some basic aggregations
prices_df['price_max'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
prices_df['price_min'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
prices_df['price_std'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
prices_df['price_mean'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# and do price normalization (min/max scaling)
prices_df['price_norm'] = prices_df['sell_price']/prices_df['price_max']

# Some items are can be inflation dependent
# and some items are very "stable"
prices_df['price_nunique'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
prices_df['item_nunique'] = prices_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')



【メモ】<br>
calendar_prices = calendar_df[['wm_yr_wk','month','year']]は、wm_yr_wk , month , year columnだけ抜き出し、
新しいdataframe calendar_pricesにしている。<br>
項目減らしているため、行単位で重複が発生するため、drop_duplicatesで重複削除。<br>
直後に、prices_dfへマージ。<br>
merge_by_concatを使っていないが、prices_dfとcalender_pricesで、'wm_yr_wk'が同じ行をprices_dfに追加している。<br>
追加しているのは、calenter_pricesの'wm_yr_wk','month','year'だけ。<br>
mergeは２つのtableを結合して、好きなtableにするときによく使う。<br>
他には、正規化されたdb(SQL等)から、意図したtableを作るときにも使う。<br>

In [ ]:
calendar_df.head()

In [ ]:
# I would like some "rolling" aggregations
# but would like months and years as "window"
calendar_prices = calendar_df[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')




In [ ]:
calendar_prices.head()

In [ ]:
del calendar_prices

【メモ】<br>
次も、新しいcolumn(特徴量)を追加している。<br>
transformについてはこちら。<br>
https://qiita.com/greenteabiscuit/items/132e0f9b1479926e07e0

In [ ]:
# Now we can add price "momentum" (some sort of)
# Shifted by week 
# by month mean
# by year mean
prices_df['price_momentum'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_m'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

del prices_df['month'], prices_df['year']

In [ ]:
prices_df.head()

In [ ]:
########################### Merge prices and save part 2
#################################################################################
print('Merge prices and save part 2')

# Merge Prices
original_columns = list(grid_df)
grid_df = grid_df.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')


In [ ]:
grid_df.head()

[col for col ...]は、original_columnsとgrid_dfのコラムのandをとっているように見える。<br>
ともにset()にしてAndしたほうがわかりやすそう。

【メモ】<br>
grid_dfの内容を一旦酢漬け。<br>
前に酢漬けしたものを読み直している<br>
メモリ使用量を減らす施策かな。メモリが豊富なら、２つのdataframeを使えばいいだけ。

In [ ]:
keep_columns = [col for col in list(grid_df) if col not in original_columns]
grid_df = grid_df[MAIN_INDEX+keep_columns]
grid_df = reduce_mem_usage(grid_df)

# Safe part 2
grid_df.to_pickle('grid_part_2.pkl')
print('Size:', grid_df.shape)

# We don't need prices_df anymore
del prices_df

# We can remove new columns
# or just load part_1
grid_df = pd.read_pickle('grid_part_1.pkl')

【メモ】<br>
ここから、データを整えています。
astypeで、dataframeのデータ型をcolumnごとに指定し、memory使用量を減らしています。

In [ ]:
########################### Merge calendar
#################################################################################
grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
icols = ['date',
         'd',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

# Minify data
# 'snap_' columns we can convert to bool or int8
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')



【メモ】<br>
to_datetimeは、日付表示をdatetimeオブジェクトにしている。<br>
datetimeオブジェクトは２つの日時の演算ができたり比較ができるので、便利で良く使います。<br>

In [ ]:
# Convert to DateTime
grid_df['date'] = pd.to_datetime(grid_df['date'])

# Make some features from date
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

# Remove date
del grid_df['date']

In [ ]:
########################### Save part 3 (Dates)
#################################################################################
print('Save part 3')

# Safe part 3
grid_df.to_pickle('grid_part_3.pkl')
print('Size:', grid_df.shape)

# We don't need calendar_df anymore
del calendar_df
del grid_df

In [ ]:
########################### Some additional cleaning
#################################################################################

## Part 1
# Convert 'd' to int
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# Remove 'wm_yr_wk'
# as test values are not in train set
del grid_df['wm_yr_wk']
grid_df.to_pickle('grid_part_1.pkl')

del grid_df

In [ ]:
########################### Summary
#################################################################################

# Now we have 3 sets of features
grid_df = pd.concat([pd.read_pickle('grid_part_1.pkl'),
                     pd.read_pickle('grid_part_2.pkl').iloc[:,2:],
                     pd.read_pickle('grid_part_3.pkl').iloc[:,2:]],
                     axis=1)
                     
# Let's check again memory usage
print("{:>20}: {:>8}".format('Full Grid',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
print('Size:', grid_df.shape)

# 2.5GiB + is is still too big to train our model
# (on kaggle with its memory limits)
# and we don't have lag features yet
# But what if we can train by state_id or shop_id?
state_id = 'CA'
grid_df = grid_df[grid_df['state_id']==state_id]
print("{:>20}: {:>8}".format('Full Grid',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
#           Full Grid:   1.2GiB

store_id = 'CA_1'
grid_df = grid_df[grid_df['store_id']==store_id]
print("{:>20}: {:>8}".format('Full Grid',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
#           Full Grid: 321.2MiB

# Seems its good enough now
# In other kernel we will talk about LAGS features
# Thank you.

In [ ]:
########################### Final list of features
#################################################################################
grid_df.info()